In [1]:
import pymysql
from itertools import chain

In [2]:
import pymysql.cursors

# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='niu950607'
                            )

#From our connection we need a cursor, which acts as our interface into the database
cur = connection.cursor()

In [3]:
print(connection)

In [ ]:
cur.execute("CREATE DATABASE ZomatoDB")

In [4]:
cur.execute('SHOW DATABASES')
for db in cur:
    print(db)

('information_schema',)
('mysql',)
('performance_schema',)
('sys',)
('zomatodb',)


In [5]:
# Connect to the database
connection = pymysql.connect(host='localhost',
                             user='root',
                             password='niu950607',
                            database='ZomatoDB')

#From our connection we need a cursor, which acts as our interface into the database
cur = connection.cursor()

In [30]:
drop_table = """DROP TABLE IF EXISTS main_table;"""

cur.execute(drop_table)
connection.commit()

In [31]:
# Create main table that contains all dataset
# Coordinates will be added separately for the moment
make_main_table = """CREATE TABLE main_table(
                        Restaurant_ID INT PRIMARY KEY,
                        Restaurant_Name VARCHAR(255),
                        Country_Code INT,
                        City VARCHAR(255),
                        Address VARCHAR(255),
                        Longitude DOUBLE,
                        Latitude DOUBLE
                        );"""
                            
cur.execute(make_main_table)
connection.commit()

In [6]:
cur.execute('SHOW TABLES')
for tb in cur:
    print(tb)

('countrycode',)
('cuisines',)
('evaluation',)
('locality',)
('main_table',)
('ratings',)


### First, Creating the Main Table that contains the information of each restaurants with unique Restaurant ID. 

In [7]:
import pandas as pd
import numpy as np

df = pd.read_csv('zomato.csv', encoding='latin1')
df.head(3)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [9]:
zomato_df = df.dropna()
zomato_df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [10]:
zomato_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9542 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9542 non-null   int64  
 1   Restaurant Name       9542 non-null   object 
 2   Country Code          9542 non-null   int64  
 3   City                  9542 non-null   object 
 4   Address               9542 non-null   object 
 5   Locality              9542 non-null   object 
 6   Locality Verbose      9542 non-null   object 
 7   Longitude             9542 non-null   float64
 8   Latitude              9542 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9542 non-null   int64  
 11  Currency              9542 non-null   object 
 12  Has Table booking     9542 non-null   object 
 13  Has Online delivery   9542 non-null   object 
 14  Is delivering now     9542 non-null   object 
 15  Switch to order menu 

In [11]:
# We forgot to format table booking, online delivery and switch to order menu as booleans:
d = {'Yes': True, 'No':False}
zomato_df['Has Table booking'] = zomato_df['Has Table booking'].map(d)
zomato_df['Has Online delivery'] = zomato_df['Has Online delivery'].map(d)
zomato_df['Is delivering now'] = zomato_df['Is delivering now'].map(d)
zomato_df['Switch to order menu'] = zomato_df['Switch to order menu'].map(d)

<ipython-input-11-da8807c71168>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zomato_df['Has Table booking'] = zomato_df['Has Table booking'].map(d)
<ipython-input-11-da8807c71168>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zomato_df['Has Online delivery'] = zomato_df['Has Online delivery'].map(d)
<ipython-input-11-da8807c71168>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [24]:
zomato_dataframe = zomato_df[['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address', 'Longitude', 'Latitude']]
zomato_dataframe.head(3)

,Restaurant ID,Restaurant Name,Country Code,City,Address,Longitude,Latitude
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...",121.027535,14.565443
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...",121.014101,14.553708
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...",121.056831,14.581404


In [25]:
#zomato_dataframe['Restaurant ID'] = zomato_dataframe['Restaurant ID'].astype(int)
zomato_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9542 entries, 0 to 9550
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Restaurant ID    9542 non-null   int64  
 1   Restaurant Name  9542 non-null   object 
 2   Country Code     9542 non-null   int64  
 3   City             9542 non-null   object 
 4   Address          9542 non-null   object 
 5   Longitude        9542 non-null   float64
 6   Latitude         9542 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 596.4+ KB


In [33]:
tuples = [tuple(x) for x in zomato_dataframe.to_numpy()]

sqlFormula = """INSERT INTO main_table 
                    (
                    Restaurant_ID,
                    Restaurant_Name,
                    Country_Code,
                    City,
                    Address,
                    Longitude,
                    Latitude
                    ) 
                    VALUES (
                    %s, %s, %s, %s, %s, %s, %s
                    );""" 
#%s placeholders

In [34]:
cur.executemany(sqlFormula, tuples)
connection.commit()

### Secondly, create a table that contains the country-code information

In [26]:
code_df = pd.read_excel('Country-Code.xlsx')
code_df.head(3)

,Country Code,Country
0,1,India
1,14,Australia
2,30,Brazil


In [27]:
# Create country-code that contains all dataset
# Coordinates will be added separately for the moment
make_countrycode_table = """CREATE TABLE countrycode(
                        Country_code INT PRIMARY KEY,
                        Country VARCHAR(255)
                        );"""
                            
cur.execute(make_countrycode_table)
connection.commit()

In [29]:
tuples_country = [tuple(x) for x in code_df.to_numpy()]

sqlFormula_country = """INSERT INTO countrycode 
                    (
                    Country_code,
                    Country
                    ) 
                    VALUES (
                    %s, %s
                    );""" 
#%s placeholders

cur.executemany(sqlFormula_country, tuples_country)
connection.commit()

### Next, we create a Cuisine table that contains the cuisine information for each restaurant

In [17]:
# return list from series of comma-separated strings
def chainer(s):
  return list(chain.from_iterable(s.str.split(',')))

# calculate lengths of splits
lens = zomato_df['Cuisines'].str.split(',').map(len)

# create new dataframe, reapeting or chaining as appropiate
res = pd.DataFrame({
    'Restaurant ID': np.repeat(zomato_df['Restaurant ID'], lens),
    'Cuisines':chainer(zomato_df['Cuisines'])
})

res_cuisine = res.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
res_cuisine.head()

,Restaurant ID,Cuisines
0,6317637,French
0,6317637,Japanese
0,6317637,Desserts
1,6304287,Japanese
2,6300002,Seafood


In [18]:
drop_table_cu = """DROP TABLE IF EXISTS cuisines;"""

cur.execute(drop_table_cu)
connection.commit()

In [19]:
make_cuisines_table = """CREATE TABLE cuisines(
                            Restaurant_ID INT,
                            Cuisine_type VARCHAR(255)
                            );
                            """

cur.execute(make_cuisines_table)
connection.commit()

In [20]:
tuples_cu = [tuple(x) for x in res_cuisine.to_numpy()]

sqlFormula_cu = """INSERT INTO cuisines 
                    (
                    Restaurant_ID,
                    Cuisine_type
                    ) 
                    VALUES (
                    %s, %s
                    );""" 
#%s placeholders

cur.executemany(sqlFormula_cu, tuples_cu)
connection.commit()

In [21]:
alter_cuisine = """
                ALTER TABLE cuisines ADD Cuisine_ID INT AUTO_INCREMENT PRIMARY KEY AFTER Restaurant_ID;
                """

cur.execute(alter_cuisine)
connection.commit()

### Then we create another table that contains the locality information for each restaurant

In [106]:
# calculate lengths of splits
lens_locality = zomato_df['Locality'].str.split(',').map(len)

# create new dataframe, reapeting or chaining as appropiate
res_first = pd.DataFrame({
    'Restaurant ID': np.repeat(zomato_df['Restaurant ID'], lens_locality),
    'Locality':chainer(zomato_df['Locality'])
})

res_locality = res_first.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
res_locality.head(10)

,Restaurant ID,Locality
0,6317637,Century City Mall
0,6317637,Poblacion
0,6317637,Makati City
1,6304287,Little Tokyo
1,6304287,Legaspi Village
1,6304287,Makati City
2,6300002,Edsa Shangri-La
2,6300002,Ortigas
2,6300002,Mandaluyong City
3,6318506,SM Megamall


In [107]:
drop_table_locality = """DROP TABLE IF EXISTS locality;"""

cur.execute(drop_table_locality)
connection.commit()

In [108]:
make_locality_table = """CREATE TABLE locality(
                            Restaurant_ID INT,
                            Locality VARCHAR(255)
                            );
                            """

cur.execute(make_locality_table)
connection.commit()

In [109]:
tuples_locality = [tuple(x) for x in res_locality.to_numpy()]

sqlFormula_locality = """INSERT INTO locality (
                    Restaurant_ID,
                    locality
                    ) 
                    VALUES (
                    %s, %s
                    );""" 
#%s placeholders

cur.executemany(sqlFormula_locality, tuples_locality)
connection.commit()

In [110]:
alter_locality = """
                ALTER TABLE locality ADD Locality_ID INT AUTO_INCREMENT PRIMARY KEY AFTER Restaurant_ID;
                """

cur.execute(alter_locality)
connection.commit()

### We create another table called Rating table that contains the rating information for each restaurant

In [12]:
zomato_rate = zomato_df[['Restaurant ID', 'Aggregate rating',
                    'Rating color',
                    'Rating text',
                    'Votes']]
zomato_rate.head()

,Restaurant ID,Aggregate rating,Rating color,Rating text,Votes
0,6317637,4.8,Dark Green,Excellent,314
1,6304287,4.5,Dark Green,Excellent,591
2,6300002,4.4,Green,Very Good,270
3,6318506,4.9,Dark Green,Excellent,365
4,6314302,4.8,Dark Green,Excellent,229


In [13]:
drop_table_rating = """DROP TABLE IF EXISTS ratings;"""

cur.execute(drop_table_rating)
connection.commit()

In [14]:
make_rating_table = """CREATE TABLE ratings (
                            Restaurant_ID INT,
                            Rating FLOAT, 
                            Rating_color VARCHAR(255),
                            Rating_text VARCHAR(255),
                            Votes FLOAT
                            );"""

cur.execute(make_rating_table)
connection.commit()

In [15]:
tuples_rate = [tuple(x) for x in zomato_rate.to_numpy()]

sqlFormula_rate = """INSERT INTO ratings (
                    Restaurant_ID,
                    Rating,
                    Rating_color,
                    Rating_text,
                    Votes
                    ) 
                    VALUES (
                    %s, %s, %s, %s, %s
                    );""" 
#%s placeholders

cur.executemany(sqlFormula_rate, tuples_rate)
connection.commit()

In [16]:
alter_rate = """
                ALTER TABLE ratings ADD Rating_ID INT AUTO_INCREMENT PRIMARY KEY AFTER Restaurant_ID;
                """

cur.execute(alter_rate)
connection.commit()

### We create the fourth table called evaluation table that contains the evaluation information for each restaurant

In [24]:
make_evaluation_table = """CREATE TABLE evaluation(
                                Restaurant_ID INT,
                                Table_booking BOOL,
                                Online_delivery BOOL,
                                Now_delivering BOOL,
                                Switch_menu BOOL,
                                Price_range INT
                                );"""
cur.execute(make_evaluation_table)
connection.commit()

In [29]:
zomato_evaluation = zomato_df[['Restaurant ID',
                               'Has Table booking', 
                               'Has Online delivery',
                               'Is delivering now', 
                               'Switch to order menu',
                               'Price range']]
zomato_evaluation.head()

,Restaurant ID,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range
0,6317637,True,False,False,False,3
1,6304287,True,False,False,False,3
2,6300002,True,False,False,False,4
3,6318506,False,False,False,False,4
4,6314302,True,False,False,False,4


In [30]:
tuples_evaluation = [tuple(x) for x in zomato_evaluation.to_numpy()]

sqlFormula_evaluation = """INSERT INTO evaluation (
                    Restaurant_ID,
                    Table_booking,
                    Online_delivery,
                    Now_delivering,
                    Switch_menu,
                    Price_range
                    ) 
                    VALUES (
                    %s, %s, %s, %s, %s, %s
                    );""" 
#%s placeholders

cur.executemany(sqlFormula_evaluation, tuples_evaluation)
connection.commit()

In [31]:
alter_evaluation = """
                ALTER TABLE evaluation ADD Evaluation_ID INT AUTO_INCREMENT PRIMARY KEY AFTER Restaurant_ID;
                """

cur.execute(alter_evaluation)
connection.commit()